In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot  as plt
from sklearn import tree
from collections import Counter
import sklearn.model_selection
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from scipy import stats
from sklearn.model_selection import cross_val_predict, train_test_split, cross_val_score, RepeatedKFold, KFold, cross_validate, GridSearchCV
from sklearn.svm import LinearSVC, SVC
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import confusion_matrix, precision_recall_curve, make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from numpy import mean, std
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
import warnings
from sklearn.cluster import KMeans, AgglomerativeClustering
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from pathlib import Path
import xgboost 
from sklearn.utils import resample
from imblearn.under_sampling import RandomUnderSampler
from Analysis_functions_ensemble import *
pd.set_option('display.max_colwidth', None)
from itertools import repeat
warnings.filterwarnings('ignore')

[NbConvertApp] Converting notebook heavy_Analysis_functions.ipynb to script
[NbConvertApp] Writing 17710 bytes to heavy_Analysis_functions.py


Creating lists for table with final results at the very end, 4 metrics used for models evaluation:
* precision
* recall
* f1 score
* accuracy
* specificity

# all features
train-test split, 20% of samples are test, seed is not random

proportions between 1 and 0 class are the same in both test and train sets

waves = ['1', '2', '3', '4', '5']
names = ['drop', 'stay']
wave_list = []
for wave in waves:
    
    df = pd.read_csv('../../data/data_online_personal_w' + wave + '.csv')
    political_data = pd.read_csv('../../data/data_online_political_w' + wave + '.csv')
    
    df = concat_df(df, political_data)
    y = df['panelpat']
    dte = df.filter(like='_dte', axis=1)
    X = df.drop(['panelpat', 'id', dte.columns[0]], axis=1)
    cols = X.columns
    print(' ')
    print('WAVE: ', wave)
    print(Counter(y))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    analysis_all_models(X_train, y_train, X_test, y_test, names)
    wave_list.append(wave)

## personal and engineered features + gender/age_group interaction

In [3]:
waves = ['1', '2', '3', '4', '5']
names = ['drop', 'stay']
for wave in waves:
    
    df = pd.read_csv('../../data/data_online_personal_w' + wave + '.csv')
    political_data = pd.read_csv('../../data/data_online_political_w' + wave + '.csv')
    
    engineered = political_data.filter(regex='whether_dropped_before|inconsistency|bad_quality|weekend|know_politicians_ratio|same_agree_resp|political_interest|responded|dont_know_percentage_mean|days_to_respond')
    y = df['panelpat']
    #dte = df.filter(like='_dte', axis=1)
    X = df.drop(['panelpat'], axis=1) #, dte.columns[0]
    X = pd.concat([X, engineered], axis=1)
    cols = X.columns
    print(' ')
    
    print('WAVE: ', wave)
    print('Personal and engineered features:')
    print(Counter(y))
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    #undersample = RandomUnderSampler(sampling_strategy='majority')
    #X_train, y_train = undersample.fit_resample(X_train, y_train)
    analysis_all_models(X_train, y_train, X_test, y_test, names)    
    
    age_dummies = X['age_group -sd2x2']
    age_dummies = pd.get_dummies(age_dummies, columns=['age_group -sd2x2'], dummy_na=True, prefix_sep='_')
    # https://stackoverflow.com/questions/42969545/interactions-between-dummies-variables-in-python
    gender = X.columns[X.columns.str.startswith('GENDER')]
    X = pd.concat([X, age_dummies], axis=1)
    age_group = X.filter(like='age_group', axis=1)

    for col1 in gender:
        for col2 in age_group:
            X[col2 + '_' + col1.split('_')[1]] = X[col1].mul(X[col2])
        
    for i in age_group:
        X = X.drop(i, axis=1)
    for i in gender:
        X = X.drop(i, axis=1)
    
    cols = X.columns
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    #undersample = RandomUnderSampler(sampling_strategy='majority')
    #X_train, y_train = undersample.fit_resample(X_train, y_train)
    print(' ')
    print('Personal and engineered features + gender/age_group interaction')
    analysis_all_models(X_train, y_train, X_test, y_test, names)    

 
WAVE:  1
Personal and engineered features:
Counter({1: 2839, 0: 777})
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Random_forest_best:  {'n_estimators': 500, 'min_samples_split': 20, 'min_samples_leaf': 5, 'max_depth': 200, 'bootstrap': True}
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Fitting 3 folds for each of 50 candidates, totalling 150 fits
 
Personal and engineered features + gender/age_group interaction
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Random_forest_best:  {'n_estimators': 500, 'min_samples_split': 20, 'min_samples_leaf': 5, 'max_depth': 100, 'bootstrap': True}
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Fitting 3 folds for each of 50 candidates, total

In [25]:
choose_data = ['Pers_engin_features'] * 4 + ['Pers_engin_features_gender_age'] * 4
choose_data *= 5
wave_list = [x for item in waves for x in repeat(item, 8)]
clf = ['XGBoost_RS', 'XGBoost_default', 'RF_default', 'RF_random_search']*10
results = pd.DataFrame(
    {'data': choose_data,
     'wave': wave_list,
     'clf': clf,
     'class_imbalance': class_imbalance_final,
     'recall_stay': recall_stay_final,
     'recall_drop': recall_drop_final,
     'accuracy': accuracy_final
    }).round(decimals=3)

In [26]:
results

,data,wave,clf,class_imbalance,recall_stay,recall_drop,accuracy
0,Pers_engin_features,1,XGBoost_RS,0.273,0.846,0.171,0.701
1,Pers_engin_features,1,XGBoost_default,0.273,0.801,0.209,0.674
2,Pers_engin_features,1,RF_default,0.273,0.994,0.003,0.781
3,Pers_engin_features,1,RF_random_search,0.273,0.834,0.227,0.704
4,Pers_engin_features_gender_age,1,XGBoost_RS,0.273,0.862,0.193,0.718
5,Pers_engin_features_gender_age,1,XGBoost_default,0.273,0.793,0.242,0.675
6,Pers_engin_features_gender_age,1,RF_default,0.273,0.994,0.008,0.783
7,Pers_engin_features_gender_age,1,RF_random_search,0.273,0.831,0.227,0.701
8,Pers_engin_features,2,XGBoost_RS,0.155,0.889,0.145,0.788
9,Pers_engin_features,2,XGBoost_default,0.155,0.873,0.145,0.775


with balancing only the best result we get for recall for drop class is up to 0.4

## X - only personal features, y - panelpat

for wave in waves:
    
    df = pd.read_csv('../../data/data_online_personal_w' + wave + '.csv')
    
    y = df['panelpat']
    #dte = df.filter(like='_dte', axis=1)
    X = df.drop(['panelpat'], axis=1) #, dte.columns[0]
    cols = X.columns
    print(' ')
    print('WAVE: ', wave)
    print(Counter(y))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    #undersample = RandomUnderSampler(sampling_strategy='majority')
    #X_train, y_train = undersample.fit_resample(X_train, y_train)
    analysis_all_models(X_train, y_train, X_test, y_test, names)
    

choose_y = ['y - panelpat: all features'] * 25 + ['y - panelpat: pers+engin'] * 25 + ['y - panelpat: pers+engin, age_gender'] * 25 + ['y - panelpat: personal only'] * 25
wave_list = [x for item in waves for x in repeat(item, 5)]*4

final_results = pd.DataFrame(zip(choose_y,
                                 precision_stay_final, 
                                 precision_drop_final, 
                                 recall_stay_final, 
                                 recall_drop_final, 
                                 recall_general_final,
                                 accuracy_final,
                                 clf_list, 
                                 wave_list,
                                 class_imbalance_final)).T

final_results_test = pd.DataFrame(zip(choose_y,
                                 precision_stay_test, 
                                 precision_drop_test, 
                                 recall_stay_test, 
                                 recall_drop_test, 
                                 recall_general_test,
                                 accuracy_test,
                                 clf_test, 
                                 wave_list,
                                 class_imbalance_test)).T
#final_results = final_results.drop([3], axis=1)
#final_results.columns = list_of_all_models
final_results.index = ['choose_y',
                      'precision_stay, %',
                      'precision_drop, %',
                      'recall_stay, %',
                      'recall_drop, %',
                      'recall_general, %',
                      'accuracy, %',
                      'clf',
                      'wave',
                      'class proportions']
final_results_test.index = final_results.index

final_results = final_results.round(3)
final_results_test = final_results_test.round(3)
#final_results = final_results.mul(100)

final_results = final_results.T.sort_values(by=['recall_drop, %'], ascending=False)
final_results_test = final_results_test.T.sort_values(by=['recall_drop, %'], ascending=False)

#final_results

final_results[['precision_stay, %', 'precision_drop, %', 'recall_stay, %',
       'recall_drop, %', 'recall_general, %', 'accuracy, %', 'class proportions']] = final_results[['precision_stay, %', 'precision_drop, %', 'recall_stay, %',
       'recall_drop, %', 'recall_general, %', 'accuracy, %', 'class proportions']].apply(pd.to_numeric)
final_results = final_results.round(decimals=3)

final_results_test[['precision_stay, %', 'precision_drop, %', 'recall_stay, %',
       'recall_drop, %', 'recall_general, %', 'accuracy, %', 'class proportions']] = final_results_test[['precision_stay, %', 'precision_drop, %', 'recall_stay, %',
       'recall_drop, %', 'recall_general, %', 'accuracy, %', 'class proportions']].apply(pd.to_numeric)
final_results_test = final_results_test.round(decimals=3)

from pivottablejs import pivot_ui
from IPython.display import HTML

pivot_ui(final_results, outfile_path='../../results_html/online_y_panelpat_cv.html')
HTML('online_y_panelpat_cv.html')

pivot_ui(final_results_test, outfile_path='../../results_html/online_y_panelpat_test.html')
HTML('online_y_panelpat_test.html')